## Creating a list of domains pairs we need to compute similarity - only within clans

In [1]:
import pandas as pd
import numpy as np
import os
import subprocess
import pickle

In [3]:
#Reading clan similarity pairs
curr_dir = !pwd
pfam_version = "31"
sim_pairs_df = pd.read_csv(curr_dir[0]+"/../10.Prediction/domains_similarity/pfam-v"+pfam_version+"/domains_pairs_for_hhalign.csv", sep=",", index_col=0)

In [6]:
### Get the domains used in pipeline
domains_th = "10"
with open(curr_dir[0]+"/../5.domains_stats/pfam-v"+pfam_version+"/filtered"+domains_th+"_list.pik", 'rb') as handle:
    filtered_domains_list = pickle.load(handle)
filtered_domains_list.sort()

with open(curr_dir[0]+"/../7.InteracDome_targetVariable/domains_ligands_propensity_dict.pik", 'rb') as handle:
    binding_props_dict = pickle.load(handle)
    
domains_prop_intersection = []
for domain in filtered_domains_list:
    if (domain in binding_props_dict.keys()):
        domains_prop_intersection.append(domain)
print "number of domains = "+str(len(domains_prop_intersection))

number of domains = 443


### Finding similarity pairs between the processed domains and the pipeline domains and processed domains themselves

In [7]:
#Reading the test domains that weren't used in the pipeline
with open(curr_dir[0]+"/processed_domains_with_labels_not_in_pipeline.pik", 'rb') as handle:
    test_domains_list = pickle.load(handle)
test_domains_list.sort()  

In [9]:
indices_for_remove = []
for index, pair in sim_pairs_df.iterrows():
    dom1 =  pair["sim_dom1"]
    dom2 =  pair["sim_dom2"]
    
    #Remove pairs that don't have a member from the test domains
    if (dom1 not in test_domains_list and dom2 not in test_domains_list):
        indices_for_remove.append(index)
    
    #Remove pairs with a member from the test domains and 2nd member not in the pipeline or test 
    elif (dom1 in test_domains_list and dom2 not in domains_prop_intersection and dom2 not in test_domains_list):
        indices_for_remove.append(index)
    elif (dom2 in test_domains_list and dom1 not in domains_prop_intersection and dom1 not in test_domains_list):
        indices_for_remove.append(index)

In [13]:
sim_pairs_df.shape

(6851346, 2)

In [10]:
sim_pairs_df_filtered = sim_pairs_df.drop(indices_for_remove)
sim_pairs_df_filtered = sim_pairs_df_filtered.reset_index(drop=True)
sim_pairs_df_filtered.shape

(61618, 2)

In [15]:
sim_pairs_df_filtered.to_csv(curr_dir[0]+"/processed_domains_pairs_for_hhalign.csv")